In [1]:
import pandas as pd
import numpy as np
import random
from typing import List
import matplotlib.pyplot as plt

from eval_plots import plot_prediction, convert_to_datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [2]:
random.seed(123)

In [3]:
target_col_name = "GDPC1"
predictions = []

In [4]:
X_train = pd.read_csv("./data/1_art_data_prep/2017-01-01/X_train.csv", index_col=0)
X_test = pd.read_csv("./data/1_art_data_prep/2017-01-01/X_test.csv", index_col=0)
y_train = pd.read_csv("./data/1_art_data_prep/2017-01-01/y_train.csv", index_col=0)
y_test = pd.read_csv("./data/1_art_data_prep/2017-01-01/y_test.csv", index_col=0)

In [5]:
X_train = convert_to_datetime(X_train.reset_index().rename(columns={"index": "date"}), ["date"])
X_test = convert_to_datetime(X_test.reset_index().rename(columns={"index": "date"}), ["date"])
y_train = convert_to_datetime(y_train.reset_index().rename(columns={"index": "date"}), ["date"])
X_train = X_train.set_index("date")
X_test = X_test.set_index("date")
y_train = y_train.set_index("date")

X_train = X_train.sort_index()
X_test = X_test.sort_index()
y_train = y_train.sort_index()

In [6]:
oom_data = pd.read_excel("./data/US/2017-01-01.xlsx", sheet_name="other")
oom_data = oom_data.pivot(index="ReferenceDate", columns="VariableCode", values="VariableValue")

exog = oom_data[['ICSA']].sort_index().pct_change().resample('MS').sum()

/var/folders/3k/vh6dl_9j30z3n567nqndm7tw0000gp/T/ipykernel_32751/4080280868.py:4: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  exog = oom_data[['ICSA']].sort_index().pct_change().resample('MS').sum()


In [7]:
X_train = X_train.merge(exog, left_index=True, right_index=True, how="left")
X_test = X_test.merge(exog, left_index=True, right_index=True, how="left")

In [8]:
from sklearn.decomposition import FactorAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)  # Fit scaler on training data
X_test_scaled = scaler.transform(X_test)        # Apply the same scaler to test data

# Step 3: Apply Factor Analysis on the training data
fa = FactorAnalysis(n_components=5)  # Example: reduce to 5 factors
X_train_fa = fa.fit_transform(X_train_scaled)  # Fit and transform on training data
X_train_fa = pd.DataFrame(X_train_fa, index=X_train.index).add_prefix("fa_")

# Step 4: Apply the learned factor analysis transformation to the test data
X_test_fa = fa.transform(X_test_scaled)
X_test_fa = pd.DataFrame(X_test_fa, index=X_test.index).add_prefix("fa_")

In [10]:
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 3: Apply the same scaling to the test data
X_test_scaled = scaler.transform(X_test)

# Step 4: Apply PCA to the scaled training data
pca = PCA(n_components=5)  # Adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train_scaled)

# Step 5: Transform the test data using the PCA model trained on the training data
X_test_pca = pca.transform(X_test_scaled)

X_train_pca = pd.DataFrame(X_train_pca, index=X_train.index).add_prefix("fa_")
X_train_pca = pd.DataFrame(X_train_pca, index=X_train.index).add_prefix("fa_")

In [11]:
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler

# Step 2: Scale the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 3: Apply the same scaling to the test data
X_test_scaled = scaler.transform(X_test)

# Step 4: Define the autoencoder architecture
input_dim = X_train_scaled.shape[1]  # Number of features
encoding_dim = 5  # Latent space dimensionality

# Encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)

# Decoder
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Step 5: Build and compile the autoencoder
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Step 6: Train the autoencoder only on the training data
autoencoder.fit(X_train_scaled, X_train_scaled, epochs=50, batch_size=256, shuffle=True)

# Step 7: Extract the encoder part for dimensionality reduction
encoder = Model(input_layer, encoded)
X_train_encoded = encoder.predict(X_train_scaled)
X_test_encoded = encoder.predict(X_test_scaled)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - loss: 1.2829
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2816
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2802
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2789
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2776
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2764
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2751
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2738
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2726
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2714
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2701
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2689
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2678
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.2666
Epoch 15/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2654
Epoch 16/50
1/1 ━━━━━━━━━━━━━━━━━

In [12]:
X_train_encoded = pd.DataFrame(X_train_encoded, index=X_train.index).add_prefix("encoded_")
X_train_encoded = pd.DataFrame(X_train_encoded, index=X_train.index).add_prefix("encoded_")

In [13]:
from sklearn.linear_model import Ridge

model = Ridge()

model.fit(X_train_fa, y_train)
train_preds = pd.DataFrame.from_dict({
    "date": X_train_fa.index,
    "predictions": model.predict(X_train_fa).reshape(1, -1)[0],
    "actuals": y_train[target_col_name].values
})

# plt.plot(train_preds.date, train_preds.actuals, label="actual") 
# plt.plot(train_preds.date, train_preds.predictions, label="predictions")
# plt.legend();
plot_prediction(dt=train_preds.date, y_pred=train_preds.predictions, y_actual=train_preds.actuals, mode = "lines+markers", legend_position=(1, 1.075))

predictions.append(model.predict(X_train_fa)[0][0])
print(predictions[-1])

-0.038677151795409204


In [14]:
from sklearn.linear_model import Ridge

model = Ridge()

model.fit(X_train_pca, y_train)
train_preds = pd.DataFrame.from_dict({
    "date": X_train_pca.index,
    "predictions": model.predict(X_train_pca).reshape(1, -1)[0],
    "actuals": y_train[target_col_name].values
})

# plt.plot(train_preds.date, train_preds.actuals, label="actual") 
# plt.plot(train_preds.date, train_preds.predictions, label="predictions")
# plt.legend();
plot_prediction(dt=train_preds.date, y_pred=train_preds.predictions, y_actual=train_preds.actuals, mode = "lines+markers", legend_position=(1, 1.075))

predictions.append(model.predict(X_train_pca)[0][0])
print(predictions[-1])

-0.5617631132056298


In [15]:
model = Ridge()

model.fit(X_train_encoded, y_train)
train_preds = pd.DataFrame.from_dict({
    "date": X_train_encoded.index,
    "predictions": model.predict(X_train_encoded).reshape(1, -1)[0],
    "actuals": y_train[target_col_name].values
})

# plt.plot(train_preds.date, train_preds.actuals, label="actual") 
# plt.plot(train_preds.date, train_preds.predictions, label="predictions")
# plt.legend();
plot_prediction(dt=train_preds.date, y_pred=train_preds.predictions, y_actual=train_preds.actuals, mode = "lines+markers", legend_position=(1, 1.075))

predictions.append(model.predict(X_train_encoded)[0][0])
print(predictions[-1])

1.6068677
